In [1]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

[]

In [5]:
!nvcc --version

/bin/bash: nvcc: command not found


In [252]:
from platform import python_version

print(python_version())

# Should be 3.7.3 for underthesea compatibility

3.7.3


In [253]:
import pandas
import unicodedata # VN NLP took
from underthesea import word_tokenize as nlp
import re

In [254]:
# mystring = 'Bao lâu thì có hàng ạ'
# test = " ".join([str(x).strip() for x in nlp(mystring.strip())])
# test
# nlp(mystring.strip())

In [255]:
df_raw = pandas.read_csv("batch1_vi_en_chat.csv", encoding = "utf-8")
df_raw.rename(columns = {'Original':'sep_msg', 
                    'Language Correction': 'clean_msg'}, inplace=True)
df_raw.head()

,Code,sep_msg,clean_msg,Translation,Note
0,B1-VN-1,"""Chất liệu: Cotton, Polyester""","""Chất liệu: Cotton, Polyester""",It is made of Cotton and Polyester,NaN
1,B1-VN-2,Dạ sản phẩm này chỉ đi kèm 1 chậu không có giá...,"Dạ, sản phẩm này chỉ đi kèm 1 chậu không có gi...","This product appears as a free pot, it does no...",NaN
2,B1-VN-3,"""Em biết là hàng k có sẵn rồi cơ nhưng mà shop...","""Em biết là hàng không có sẵn rồi nhưng mà cửa...",I know that the product is not available right...,NaN
3,B1-VN-4,Em 65kg,Em nặng 65kg,I weigh 65kg,NaN
4,B1-VN-5,Bao lâu thì có hàng ạ.<eos>Mình ở HN,Bao lâu thì có hàng ạ?<eos>Mình ở Hà Nội.,How long does it take to have goods?<eos>I liv...,NaN


In [256]:
df = df_raw[['sep_msg', 'clean_msg']]
df.dropna(inplace=True)
df = df.applymap(lambda x: x.replace('"', ''))
df.head()

/ldap_home/yoong.thuydung/anaconda3/envs/mlt03/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sep_msg,clean_msg
0,"Chất liệu: Cotton, Polyester","Chất liệu: Cotton, Polyester"
1,Dạ sản phẩm này chỉ đi kèm 1 chậu không có giá...,"Dạ, sản phẩm này chỉ đi kèm 1 chậu không có gi..."
2,Em biết là hàng k có sẵn rồi cơ nhưng mà shop ...,Em biết là hàng không có sẵn rồi nhưng mà cửa ...
3,Em 65kg,Em nặng 65kg
4,Bao lâu thì có hàng ạ.<eos>Mình ở HN,Bao lâu thì có hàng ạ?<eos>Mình ở Hà Nội.


In [257]:
# Uncomment if want to remove the phonetics
# Not recommended because ascii unable to decode certain VN characters

#df.sep_msg = df.sep_msg.apply(lambda val: unicodedata.normalize('NFKD', str(val)).encode('ascii', 'ignore').decode())
#df.clean_msg = df.clean_msg.apply(lambda val: unicodedata.normalize('NFKD', str(val)).encode('ascii', 'ignore').decode())
#df.head()

In [258]:
# Uncomment if want to test with only a small portion of the data

#df = df.iloc[:1000]
#df

In [259]:
def remove_emoji(string): # new emoji updates https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


def remove_email(line):
    SUPPORT_EMAIL = r"support@shopee\.\S{2}"
    line = re.sub(SUPPORT_EMAIL, "SUPPORT_EMAIL", line)
    line = re.sub(
        r"(\b[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+\b)", "EMAIL", line
    )
    return line


def remove_url(line):
    line = re.sub(r"https?://.\S+", "URL", line)
    return line

def replace_wordnum(line): #mixture of word and number, eg. H219DW
    return re.sub(r'\w*\d\w*', 'WORDNUM', line)

def split_number(line):
    return re.sub(r"(\d+)([^\d]+)", r"\1 \2", line)

def replace_number(line: str) -> str:
    return re.sub(r"\s[0-9]+\s", " NUM ", line)

def lowercase(line):
    return line.lower()

def preprocess(line):
    return replace_wordnum(replace_number(split_number(remove_url(remove_email(remove_emoji(lowercase(line)))))))

In [260]:
#text = 'qwkjehl ") { []} w \dfdd'
#preprocess(mystring)
#replace_wordnum(mystring)
#replace_number(split_number(mystring))
#replace_wordnum(replace_number(split_number(mystring)))

In [261]:
#dftest = df.iloc[45:55]
df['sep_msg'] = df['sep_msg'].apply(preprocess)
df['clean_msg'] = df['clean_msg'].apply(preprocess)
df.head()

,sep_msg,clean_msg
0,"chất liệu: cotton, polyester","chất liệu: cotton, polyester"
1,dạ sản phẩm này chỉ đi kèm NUM chậu không có ...,"dạ, sản phẩm này chỉ đi kèm NUM chậu không có..."
2,em biết là hàng k có sẵn rồi cơ nhưng mà shop ...,em biết là hàng không có sẵn rồi nhưng mà cửa ...
3,em NUM kg,em nặng NUM kg
4,bao lâu thì có hàng ạ.<eos>mình ở hn,bao lâu thì có hàng ạ?<eos>mình ở hà nội.


In [262]:
#len('nghiêng')

In [263]:

alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace(r'\n',". ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<eos> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<eos> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace("\\","")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    if "(" in text: text = text.replace("(","")
    if ")" in text: text = text.replace(")","")
    if "[" in text: text = text.replace("[","")
    if "]" in text: text = text.replace("]","")
    text = text.replace(".",".<eos>")
    text = text.replace("?","?<eos>")
    text = text.replace("!","!<eos>")
    text = text.replace("<prd>",".")
    sentences = text.split("<eos>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [264]:
counter = 0
with open("processed_vn_batch1.txt", 'w') as f:
    for y, row in df.iterrows():
        orig = str(row.sep_msg).replace(r'\n'," ")
        clean = str(row.clean_msg).replace(r'\n'," ")
        #if clean == "nan":
            #orig = ""
        orig = orig.split("<eos>")
        clean = clean.split("<eos>")
        if not len(orig) == len(clean):
            counter +=1
            #print(orig)
            #print(clean)
            orig = split_into_sentences(row.sep_msg)
            clean = split_into_sentences(row.clean_msg)
        if not len(orig) == len(clean):
            continue
        for i in range(len(orig)):
            src = " ".join([str(x).strip() for x in nlp(orig[i].strip())])
            #print(src)
            trg = " ".join([str(x).strip() for x in nlp(clean[i].strip())])
            #print(clean)
            if len(src) < 3:
                continue
            if len(trg) < 3:
                continue
            f.write(src)
            f.write('\t')
            f.write(trg)
            f.write('\n')
            
print(counter)

600


In [265]:
with open('processed_vn_batch1.txt', 'r') as f:
    lines = [line for line in f]
    n = len(lines)
    train1 = lines[0:int(0.8*n)]
    train = lines[0:int(0.8*len(train1))]
    valid = lines[int(-0.2*len(train1)):]
    test = lines[int(-0.2*n):]
    
    for line in train:
        content = line.split('\t')
        src = content[0]
        trg = content[1]
        with open('src-train.txt', 'a') as src_file:
            src_file.write(src)
            src_file.write('\n')
        with open('tgt-train.txt', 'a') as trg_file:
            trg_file.write(trg)
            #trg_file.write('\n')
            
    for line in test:
        content = line.split('\t')
        src = content[0]
        trg = content[1]
        with open('src-test.txt', 'a') as src_file:
            src_file.write(src)
            src_file.write('\n')
        with open('tgt-test.txt', 'a') as trg_file:
            trg_file.write(trg)
            #trg_file.write('\n')
            
    for line in valid:
        content = line.split('\t')
        src = content[0]
        trg = content[1]
        with open('src-val.txt', 'a') as src_file:
            src_file.write(src)
            src_file.write('\n')
        with open('tgt-val.txt', 'a') as trg_file:
            trg_file.write(trg)
            #trg_file.write('\n')